In [55]:
import ssl
import nltk
import certifi
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

In [56]:
# Load the pre-trained model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a sentiment-analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [57]:
sentiment_analyzer('Ursula')

[{'label': 'LABEL_1', 'score': 0.6010197997093201}]

In [58]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving Mock_test_manual1.csv to Mock_test_manual1.csv
User uploaded file "Mock_test_manual1.csv" with length 46442 bytes


In [59]:
conversations = pd.read_csv("Mock_test_manual1.csv")
conversations.head(10)

,Unnamed: 0,_id,created_at_datetime,text,in_reply_to_status_id,in_reply_to_user_id,lang,user,actual_sent,cleaned_text,conv_starter
0,5,1131182474380423168,2019-05-22 12:58:12,Forcibly took my daughter's bag from her to ch...,NaN,NaN,en,1604219274,negative,forcibly took my daughters bag from her to che...,1131182474380423168
1,6,1131187978771025921,2019-05-22 13:20:05,@addblackman We understand that you would pref...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we understand that you would pref...,1131182474380423168
2,7,1131188089328734209,2019-05-22 13:20:31,@addblackman We do understand that some passen...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we do understand that some passen...,1131182474380423168
3,8,1131188303821180928,2019-05-22 13:21:22,"@addblackman Keys, medicines or your laptop ma...",1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman keys medicines or your laptop may...,1131182474380423168
4,22,1131191763912282112,2019-05-22 13:35:07,"@KLM Hi, any chance you can retrieve an old bo...",NaN,5.637714e+07,en,4806232401,positive,@klm hi any chance you can retrieve an old boa...,1131191763912282112
5,23,1131197267493445633,2019-05-22 13:56:59,"@stargirlTT Unfortunately, we cannot send a bo...",1.131192e+18,4.806232e+09,en,56377143,neutral,@stargirltt unfortunately we cannot send a boa...,1131191763912282112
6,24,1131197643626110977,2019-05-22 13:58:29,@KLM No problem. Thanks for getting back to me...,1.131197e+18,5.637714e+07,en,4806232401,positive,@klm no problem thanks for getting back to me ...,1131191763912282112
7,25,1131198945751982081,2019-05-22 14:03:39,@klm last time I fly with you. Yesterday you a...,NaN,5.637714e+07,en,15168078,negative,@klm last time i fly with you yesterday you ac...,1131198945751982081
8,26,1131200135608885249,2019-05-22 14:08:23,@astonbowles If all your flights are booked in...,1.131199e+18,1.516808e+07,en,56377143,neutral,@astonbowles if all your flights are booked in...,1131198945751982081
9,27,1131215028512874498,2019-05-22 15:07:34,@KLM The next available flight is 24hrs later ...,1.131200e+18,5.637714e+07,en,15168078,negative,@klm the next available flight is 24hrs later ...,1131198945751982081


In [60]:
# Configure SSL context to use certifis CA bundle
ssl_context = ssl.create_default_context(cafile=certifi.where())
ssl._create_default_https_context = lambda: ssl_context

# Download the vader_lexicon data
nltk.download('vader_lexicon')

analyser = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [61]:
# Data Cleaning (only remove URLs)
# Overrides the existing column "cleaned_text"

# Function to clean tweet text without removing special characters
def clean_tweet_text(text):
    # Remove URLs
    text = str(text)
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    return text

# Apply cleaning function to the text column
conversations['clean_text'] = conversations['text'].apply(clean_tweet_text)

# conversations = conversations.drop(conversations[conversations.sentiment == "Irrelevant"].index)

conversations.head(10)

,Unnamed: 0,_id,created_at_datetime,text,in_reply_to_status_id,in_reply_to_user_id,lang,user,actual_sent,cleaned_text,conv_starter,clean_text
0,5,1131182474380423168,2019-05-22 12:58:12,Forcibly took my daughter's bag from her to ch...,NaN,NaN,en,1604219274,negative,forcibly took my daughters bag from her to che...,1131182474380423168,Forcibly took my daughter's bag from her to ch...
1,6,1131187978771025921,2019-05-22 13:20:05,@addblackman We understand that you would pref...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we understand that you would pref...,1131182474380423168,@addblackman We understand that you would pref...
2,7,1131188089328734209,2019-05-22 13:20:31,@addblackman We do understand that some passen...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we do understand that some passen...,1131182474380423168,@addblackman We do understand that some passen...
3,8,1131188303821180928,2019-05-22 13:21:22,"@addblackman Keys, medicines or your laptop ma...",1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman keys medicines or your laptop may...,1131182474380423168,"@addblackman Keys, medicines or your laptop ma..."
4,22,1131191763912282112,2019-05-22 13:35:07,"@KLM Hi, any chance you can retrieve an old bo...",NaN,5.637714e+07,en,4806232401,positive,@klm hi any chance you can retrieve an old boa...,1131191763912282112,"@KLM Hi, any chance you can retrieve an old bo..."
5,23,1131197267493445633,2019-05-22 13:56:59,"@stargirlTT Unfortunately, we cannot send a bo...",1.131192e+18,4.806232e+09,en,56377143,neutral,@stargirltt unfortunately we cannot send a boa...,1131191763912282112,"@stargirlTT Unfortunately, we cannot send a bo..."
6,24,1131197643626110977,2019-05-22 13:58:29,@KLM No problem. Thanks for getting back to me...,1.131197e+18,5.637714e+07,en,4806232401,positive,@klm no problem thanks for getting back to me ...,1131191763912282112,@KLM No problem. Thanks for getting back to me...
7,25,1131198945751982081,2019-05-22 14:03:39,@klm last time I fly with you. Yesterday you a...,NaN,5.637714e+07,en,15168078,negative,@klm last time i fly with you yesterday you ac...,1131198945751982081,@klm last time I fly with you. Yesterday you a...
8,26,1131200135608885249,2019-05-22 14:08:23,@astonbowles If all your flights are booked in...,1.131199e+18,1.516808e+07,en,56377143,neutral,@astonbowles if all your flights are booked in...,1131198945751982081,@astonbowles If all your flights are booked in...
9,27,1131215028512874498,2019-05-22 15:07:34,@KLM The next available flight is 24hrs later ...,1.131200e+18,5.637714e+07,en,15168078,negative,@klm the next available flight is 24hrs later ...,1131198945751982081,@KLM The next available flight is 24hrs later ...


In [62]:
conversations['compound'] = [analyser.polarity_scores(x)['compound'] for x in conversations['clean_text']]

In [63]:
conversations.head(10)

,Unnamed: 0,_id,created_at_datetime,text,in_reply_to_status_id,in_reply_to_user_id,lang,user,actual_sent,cleaned_text,conv_starter,clean_text,compound
0,5,1131182474380423168,2019-05-22 12:58:12,Forcibly took my daughter's bag from her to ch...,NaN,NaN,en,1604219274,negative,forcibly took my daughters bag from her to che...,1131182474380423168,Forcibly took my daughter's bag from her to ch...,0.0000
1,6,1131187978771025921,2019-05-22 13:20:05,@addblackman We understand that you would pref...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we understand that you would pref...,1131182474380423168,@addblackman We understand that you would pref...,0.7506
2,7,1131188089328734209,2019-05-22 13:20:31,@addblackman We do understand that some passen...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we do understand that some passen...,1131182474380423168,@addblackman We do understand that some passen...,-0.6619
3,8,1131188303821180928,2019-05-22 13:21:22,"@addblackman Keys, medicines or your laptop ma...",1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman keys medicines or your laptop may...,1131182474380423168,"@addblackman Keys, medicines or your laptop ma...",0.6705
4,22,1131191763912282112,2019-05-22 13:35:07,"@KLM Hi, any chance you can retrieve an old bo...",NaN,5.637714e+07,en,4806232401,positive,@klm hi any chance you can retrieve an old boa...,1131191763912282112,"@KLM Hi, any chance you can retrieve an old bo...",0.2500
5,23,1131197267493445633,2019-05-22 13:56:59,"@stargirlTT Unfortunately, we cannot send a bo...",1.131192e+18,4.806232e+09,en,56377143,neutral,@stargirltt unfortunately we cannot send a boa...,1131191763912282112,"@stargirlTT Unfortunately, we cannot send a bo...",-0.3400
6,24,1131197643626110977,2019-05-22 13:58:29,@KLM No problem. Thanks for getting back to me...,1.131197e+18,5.637714e+07,en,4806232401,positive,@klm no problem thanks for getting back to me ...,1131191763912282112,@KLM No problem. Thanks for getting back to me...,0.2183
7,25,1131198945751982081,2019-05-22 14:03:39,@klm last time I fly with you. Yesterday you a...,NaN,5.637714e+07,en,15168078,negative,@klm last time i fly with you yesterday you ac...,1131198945751982081,@klm last time I fly with you. Yesterday you a...,-0.7096
8,26,1131200135608885249,2019-05-22 14:08:23,@astonbowles If all your flights are booked in...,1.131199e+18,1.516808e+07,en,56377143,neutral,@astonbowles if all your flights are booked in...,1131198945751982081,@astonbowles If all your flights are booked in...,-0.5574
9,27,1131215028512874498,2019-05-22 15:07:34,@KLM The next available flight is 24hrs later ...,1.131200e+18,5.637714e+07,en,15168078,negative,@klm the next available flight is 24hrs later ...,1131198945751982081,@KLM The next available flight is 24hrs later ...,-0.4767


In [64]:
# Function to categorize sentiment based on score
def categorize_sentiment(row):
    score = row["compound"]

    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

# # Apply categorization function to the sentiment score column
conversations['vader'] = conversations.apply(categorize_sentiment, axis=1)

def is_equal(row):
    sentiment = row["actual_sent"]
    vader = row["vader"]

    if vader == sentiment:
        return True
    else:
        return False

conversations["vader_acc"] = conversations.apply(is_equal, axis=1)

In [65]:
conversations.head(20)

counts = conversations['vader_acc'].value_counts()
print(counts)

vader_acc
True     58
False    53
Name: count, dtype: int64


In [68]:
conversations.head(10)

,Unnamed: 0,_id,created_at_datetime,text,in_reply_to_status_id,in_reply_to_user_id,lang,user,actual_sent,cleaned_text,conv_starter,clean_text,compound,vader,vader_acc
0,5,1131182474380423168,2019-05-22 12:58:12,Forcibly took my daughter's bag from her to ch...,NaN,NaN,en,1604219274,negative,forcibly took my daughters bag from her to che...,1131182474380423168,Forcibly took my daughter's bag from her to ch...,0.0000,neutral,False
1,6,1131187978771025921,2019-05-22 13:20:05,@addblackman We understand that you would pref...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we understand that you would pref...,1131182474380423168,@addblackman We understand that you would pref...,0.7506,positive,True
2,7,1131188089328734209,2019-05-22 13:20:31,@addblackman We do understand that some passen...,1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman we do understand that some passen...,1131182474380423168,@addblackman We do understand that some passen...,-0.6619,negative,False
3,8,1131188303821180928,2019-05-22 13:21:22,"@addblackman Keys, medicines or your laptop ma...",1.131182e+18,1.604219e+09,en,56377143,positive,@addblackman keys medicines or your laptop may...,1131182474380423168,"@addblackman Keys, medicines or your laptop ma...",0.6705,positive,True
4,22,1131191763912282112,2019-05-22 13:35:07,"@KLM Hi, any chance you can retrieve an old bo...",NaN,5.637714e+07,en,4806232401,positive,@klm hi any chance you can retrieve an old boa...,1131191763912282112,"@KLM Hi, any chance you can retrieve an old bo...",0.2500,positive,True
5,23,1131197267493445633,2019-05-22 13:56:59,"@stargirlTT Unfortunately, we cannot send a bo...",1.131192e+18,4.806232e+09,en,56377143,neutral,@stargirltt unfortunately we cannot send a boa...,1131191763912282112,"@stargirlTT Unfortunately, we cannot send a bo...",-0.3400,negative,False
6,24,1131197643626110977,2019-05-22 13:58:29,@KLM No problem. Thanks for getting back to me...,1.131197e+18,5.637714e+07,en,4806232401,positive,@klm no problem thanks for getting back to me ...,1131191763912282112,@KLM No problem. Thanks for getting back to me...,0.2183,positive,True
7,25,1131198945751982081,2019-05-22 14:03:39,@klm last time I fly with you. Yesterday you a...,NaN,5.637714e+07,en,15168078,negative,@klm last time i fly with you yesterday you ac...,1131198945751982081,@klm last time I fly with you. Yesterday you a...,-0.7096,negative,True
8,26,1131200135608885249,2019-05-22 14:08:23,@astonbowles If all your flights are booked in...,1.131199e+18,1.516808e+07,en,56377143,neutral,@astonbowles if all your flights are booked in...,1131198945751982081,@astonbowles If all your flights are booked in...,-0.5574,negative,False
9,27,1131215028512874498,2019-05-22 15:07:34,@KLM The next available flight is 24hrs later ...,1.131200e+18,5.637714e+07,en,15168078,negative,@klm the next available flight is 24hrs later ...,1131198945751982081,@KLM The next available flight is 24hrs later ...,-0.4767,negative,True


In [67]:
conversations.shape

(111, 15)